<a href="https://colab.research.google.com/github/ftk1000/GraphNeuralNets/blob/main/gin_in_tf230.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
ROOT_DIR = os.path.abspath("/content")
os.chdir(ROOT_DIR)
!git clone https://github.com/calciver/Graph-Isomorphism-Networks.git

WORK_DIR = os.path.abspath("/content/Graph-Isomorphism-Networks")
os.chdir(WORK_DIR)

Cloning into 'Graph-Isomorphism-Networks'...
remote: Enumerating objects: 73, done.
remote: Total 73 (delta 0), reused 0 (delta 0), pack-reused 73
Unpacking objects: 100% (73/73), done.


In [2]:
!unzip -q dataset.zip

In [23]:
!pip install -q tensorflow-gpu   #==2.0.0-alpha0
# !pip install -r requirements.txt

     |████████████████████████████████| 320.4MB 57kB/s 


In [5]:
from platform import python_version
print(python_version())  #3.6.9

import tensorflow as tf
import numpy as np
print(tf.__version__) # 2.3.0
print(np.__version__) # 1.18.5

3.6.9
2.3.0
1.18.5


In [4]:
from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

print(tf.executing_eagerly())

True


In [6]:
import easydict
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import networkx as nx

In [7]:
#@title ### Choosing parameters for this Colaboratory runtime  { form-width: "60%", run: "auto"}
#@markdown <br>Connect to a local or hosted Colaboratory runtime by clicking the **Connect** button at the top-right.
#@markdown ---
#@markdown For all configurations, **5 GNN layers** (including the input layer) are applied, and all **MLPs have 2 layers**.
#@markdown The hyper-parameters the **authors** tuned for each dataset are:
#@markdown 1. the number of hidden units ∈ {16, 32} for bioinformatics graphs and 64 for social graphs
#@markdown 2. the batch size ∈ {32, 128}
#@markdown 3. the dropout ratio ∈ {0, 0.5} after the dense layer (Srivastava et al., 2014)
#@markdown 4. the number of epochs, i.e., a single epoch with the best cross-validation accuracy averaged over the 10 folds was selected
#@markdown The **authors'** reported training accuracy were on the following fixed hyper-parameters:
#@markdown 5 GNN layers (including the input layer), hidden units of size 64, minibatch of size 128, and 0.5 dropout ratio
#@markdown ---

Dataset = "REDDITBINARY"  #@param ["MUTAG", "COLLAB", "IMDBMULTI","NCI1","PTC","REDDITMULTI5K","IMDBBINARY" ,"PROTEINS" , "REDDITBINARY"]
Batch_Size = 64 #@param {type:"slider", min:1, max:1000, step:1}
Iterations_Per_Epoch = 50 #@param {type:"slider", min:1, max:1000, step:1}
Epochs = 5 #@param {type:"slider", min:1, max:1000, step:1}
Learning_Rate = 0.01 #@param {type:"slider", min:0, max:1, step:0.0001}
Seed = 0 #@param {type:"slider", min:0, max:10, step:1}
Fold_idx = 0 #@param ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"] {type:"raw", allow-input: true}
Num_Layers = 5 #@param {type:"slider", min:1, max:10, step:1}
Num_MLP_Layers = 2 #@param {type:"slider", min:1, max:10, step:1}
Hidden_dim = 64 #@param {type:"slider", min:1, max:300, step:1}
Final_dropout = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
Graph_Pooling_Type = "sum"  #@param ["sum", "mean", "average"]
Neighbor_Pooling_Type = "sum"  #@param ["sum", "mean", "average","max"]
Learn_Epsilon = "True"  #@param ["True", "False"]
Degree_as_tag = "True"  #@param ["True", "False"]
File_Name = "output.txt" #@param {type:"string"}

In [8]:
args = easydict.EasyDict({
    "dataset": Dataset,
    "batch_size": Batch_Size,
    "iters_per_epoch": Iterations_Per_Epoch,
    "epochs": Epochs,
    "lr": Learning_Rate,
    "seed": Seed,
    "fold_idx": Fold_idx,
    "num_layers": Num_Layers,
    "num_mlp_layers": Num_MLP_Layers,
    "hidden_dim": Hidden_dim,
    "final_dropout": Final_dropout,
    "graph_pooling_type": Graph_Pooling_Type,
    "neighbor_pooling_type": Neighbor_Pooling_Type,
    "learn_eps": Learn_Epsilon,
    'degree_as_tag': Degree_as_tag,
    'filename': File_Name
    
})

In [31]:
args

{'batch_size': 64,
 'dataset': 'REDDITBINARY',
 'degree_as_tag': 'True',
 'epochs': 5,
 'filename': 'output.txt',
 'final_dropout': 0.5,
 'fold_idx': 0,
 'graph_pooling_type': 'sum',
 'hidden_dim': 64,
 'iters_per_epoch': 50,
 'learn_eps': 'True',
 'lr': 0.01,
 'neighbor_pooling_type': 'sum',
 'num_layers': 5,
 'num_mlp_layers': 2,
 'seed': 0}

In [9]:
# import numpy as np
import random
# import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

In [33]:
class S2VGraph(object):
    def __init__(self, g, label, node_tags=None, node_features=None):
        '''
            g: a networkx graph
            label: an integer graph label
            node_tags: a list of integer node tags
            node_features: a torch float tensor, one-hot representation of the tag that is used as input to neural nets
            edge_mat: a torch long tensor, contain edge list, will be used to create torch sparse tensor
            neighbors: list of neighbors (without self-loop)
        '''
        self.label = label
        self.g = g
        self.node_tags = node_tags
        self.neighbors = []
        self.node_features = 0
        self.edge_mat = 0

        self.max_neighbor = 0


def load_data(dataset, degree_as_tag):
    '''
        dataset: name of dataset
        test_proportion: ratio of test train split
        seed: random seed for random splitting of dataset
    '''

    print('loading data')
    g_list = []
    label_dict = {}
    feat_dict = {}
    triggered = False

    
    #This 1st section of the load_data function reads every row in the text file.
    #The first row of every graph will provide you with 2 things, the number of nodes, the associated label
    #The subsequent rows represent each node, and will begin with 0
    #Represented in this order (0,number of connections, <nodes that they are connected to>)
    #The graphs are created to store each node, and to add the edge lists inside
    #Thereafter, each graph is stored into a list known as g_list
    with open('dataset/%s/%s.txt' % (dataset, dataset), 'r') as f:
        n_g = int(f.readline().strip())
        for i in range(n_g):
            row = f.readline().strip().split()
            n, l = [int(w) for w in row]
            if not l in label_dict:
                mapped = len(label_dict)
                label_dict[l] = mapped
            g = nx.Graph()
            node_tags = []
            node_features = []
            n_edges = 0
            for j in range(n):
                g.add_node(j)
                row = f.readline().strip().split()
                tmp = int(row[1]) + 2
                if tmp == len(row):
                    # no node attributes
                    row = [int(w) for w in row]
                    attr = None
                else:
                    row, attr = [int(w) for w in row[:tmp]], np.array([float(w) for w in row[tmp:]])
                    triggered = True
                if triggered:
                  print('Triggered already')
                if not row[0] in feat_dict:
                    mapped = len(feat_dict)
                    feat_dict[row[0]] = mapped
                node_tags.append(feat_dict[row[0]])

                if tmp > len(row):
                    node_features.append(attr)

                n_edges += row[1]
                for k in range(2, len(row)):
                    g.add_edge(j, row[k])

            if node_features != []:
                node_features = np.stack(node_features)
                node_feature_flag = True
            else:
                node_features = None
                node_feature_flag = False

            assert len(g) == n
            g_list.append(S2VGraph(g, l, node_tags))

            
    #This is the 2nd part of the load_data code.
    #Within each graph class, he is creating the following:
    #1. graph.neighbors <matrix> : This is similar to the matrix A, but it only contains the data of the connections.
    #2. graph.max_neighbor <int>: This is basically the maximum number of connections that arises from a single node in the graph
    #3. graph.label <int>        : This is to ensure that the labels do not skip numbers, i.e. labels <1,2,4,5> becomes <0,1,2,3>
    #4. graph.edge_mat <matrix>  : This is a transpose of the edge matrix from the graph [[nodes][connected nodes]]
            
    #add labels and edge_mat       
    for g in g_list:
        g.neighbors = [[] for i in range(len(g.g))]
        for i, j in g.g.edges():
            g.neighbors[i].append(j)
            g.neighbors[j].append(i)
            
        degree_list = []
        for i in range(len(g.g)):
            g.neighbors[i] = g.neighbors[i]  #This line appears to be redundant
            degree_list.append(len(g.neighbors[i]))         
        g.max_neighbor = max(degree_list)
        g.label = label_dict[g.label]  #All this line does, is ensure that there is no skipping of labels. The actual numerical label is just a variable.
    
        #These two lines create an extension of the edges to ensure that they are bi-directional
        edges = [list(pair) for pair in g.g.edges()]
        edges.extend([[i, j] for j, i in edges])

        deg_list = list(dict(g.g.degree(range(len(g.g)))).values())
        
        g.edge_mat = tf.transpose(tf.constant(edges))
        
        
        
    #This part gives the degree dictionary but not in the order of nodes within the dataset.txt file  
    if degree_as_tag:
        for g in g_list:
            g.node_tags = list(dict(g.g.degree).values())

    #Extracting unique tag labels   
    tagset = set([])
    for g in g_list:
        tagset = tagset.union(set(g.node_tags))
    tagset = list(tagset)
    tag2index = {tagset[i]:i for i in range(len(tagset))}
    
    
    for g in g_list:
        
        node_features = np.zeros((len(g.node_tags), len(tagset)))
        node_features[range(len(g.node_tags)), [tag2index[tag] for tag in g.node_tags]] = 1  
        g.node_features = tf.constant(node_features)
        
        
    print('# classes: %d' % len(label_dict))
    print('# maximum node tag: %d' % len(tagset))

    print("# data: %d" % len(g_list))

    return g_list, len(label_dict)

def separate_data(graph_list, seed, fold_idx):
    assert 0 <= fold_idx and fold_idx < 10, "fold_idx must be from 0 to 9."
    skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = seed)

    labels = [graph.label for graph in graph_list]
    idx_list = []
    for idx in skf.split(np.zeros(len(labels)), labels):
        idx_list.append(idx)
    
    train_idx, test_idx = idx_list[fold_idx]
    
    train_graph_list = [graph_list[i] for i in train_idx]
    test_graph_list = [graph_list[i] for i in test_idx]

    return train_graph_list, test_graph_list

In [34]:
import tensorflow as tf

class MLP(tf.keras.layers.Layer):
  def __init__(self,num_layers, hidden_dim, output_dim):
    '''
        num_layers: number of layers in the neural networks (EXCLUDING the input layer). If num_layers=1, this reduces to linear model.
        hidden_dim: dimensionality of hidden units at ALL layers
        output_dim: number of classes for prediction
    '''    
    super(MLP,self).__init__()
    
    self.linear_or_not = True #default is linear model
    self.num_layers = num_layers
    
    
    if num_layers < 1:
        raise ValueError("number of layers should be positive!")
    elif num_layers == 1:
        #Linear model
        self.linear = Linear_model(output_dim = output_dim)
    else:
        #Multi-layer model
        self.linear_or_not = False
        self.multi = Multi_model(layers = num_layers,hidden_dim = hidden_dim,output_dim = output_dim)
             
        
        

  def call(self,input_features):
    
    if self.linear_or_not:
        #If linear model
        return self.linear(input_features)
    else:
        #If MLP
        return self.multi(input_features)
    
  
class Linear_model(tf.keras.layers.Layer):
  def __init__(self, output_dim):
    super(Linear_model,self).__init__()
    self.output_layer = tf.keras.layers.Dense(units = output_dim)
    
  def call(self,input_features):
    return self.output_layer(input_features)  
    
  
class Multi_model(tf.keras.layers.Layer):
  def __init__(self,layers,hidden_dim,output_dim):
    super(Multi_model,self).__init__()
    
    self.layers = layers
    self.dense_list = []
    self.batch_list = []
    
  
    for i in range(layers-1):
      self.dense_list.append(tf.keras.layers.Dense(units = hidden_dim))
      self.batch_list.append(tf.keras.layers.BatchNormalization())
    self.dense_list.append(tf.keras.layers.Dense(units = output_dim))
    
  
    
  def call(self,input_features):
    for i in range(self.layers-1):
      densed = self.dense_list[i](input_features)
      batched = self.batch_list[i](densed)
      input_features = tf.nn.relu(batched)
    multi_result = self.dense_list[-1](input_features)
  
    return multi_result

In [35]:
import tensorflow as tf

class GraphCNN(tf.keras.Model):
    def __init__(self, num_layers, num_mlp_layers, hidden_dim, output_dim, final_dropout, learn_eps, graph_pooling_type, neighbor_pooling_type):
        '''
            num_layers: number of layers in the neural networks (INCLUDING the input layer)
            num_mlp_layers: number of layers in mlps (EXCLUDING the input layer)
            hidden_dim: dimensionality of hidden units at ALL layers
            output_dim: number of classes for prediction
            final_dropout: dropout ratio on the final linear layer
            learn_eps: If True, learn epsilon to distinguish center nodes from neighboring nodes. If False, aggregate neighbors and center nodes altogether. 
            neighbor_pooling_type: how to aggregate neighbors (mean, average, or max)
            graph_pooling_type: how to aggregate entire nodes in a graph (mean, average)
        '''

        super(GraphCNN, self).__init__()

        self.final_dropout = final_dropout
        self.num_layers = num_layers
        self.graph_pooling_type = graph_pooling_type
        self.neighbor_pooling_type = neighbor_pooling_type
        self.learn_eps = learn_eps
        self.eps = tf.Variable(tf.zeros(self.num_layers-1))

    
        ###List of MLPs
        ###List of batchnorms applied to the output of MLP (input of the final prediction linear layer)
        self.mlps = []
        self.batches = []
        self.linears = []
        self.drops = []
        
        for layer in range(self.num_layers-1):
            self.mlps.append(MLP(num_mlp_layers, hidden_dim, hidden_dim))
            self.batches.append(tf.keras.layers.BatchNormalization())
            self.linears.append(tf.keras.layers.Dense(output_dim))
            self.drops.append(tf.keras.layers.Dropout(final_dropout))
            
            
        self.linears.append(tf.keras.layers.Dense(output_dim))
        self.drops.append(tf.keras.layers.Dropout(final_dropout))
        
        

    def __preprocess_neighbors_maxpool(self, batch_graph):
        ###create padded_neighbor_list in concatenated graph

        #compute the maximum number of neighbors within the graphs in the current minibatch
        max_deg = max([graph.max_neighbor for graph in batch_graph])

        padded_neighbor_list = []
        start_idx = [0]


        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            padded_neighbors = []
            for j in range(len(graph.neighbors)):
                #add off-set values to the neighbor indices
                pad = [n + start_idx[i] for n in graph.neighbors[j]]
                #padding, dummy data is assumed to be stored in -1
                pad.extend([-1]*(max_deg - len(pad)))

                #Add center nodes in the maxpooling if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.
                if not self.learn_eps:
                    pad.append(j + start_idx[i])
                
                padded_neighbors.append(pad)
            padded_neighbor_list.extend(padded_neighbors)

        
        return tf.constant(padded_neighbor_list)


    def __preprocess_neighbors_sumavepool(self, batch_graph):
        ###create block diagonal sparse matrix

        edge_mat_list = []
        start_idx = [0]
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            edge_mat_list.append(graph.edge_mat + start_idx[i])

        Adj_block_idx = tf.concat(edge_mat_list,1)
        Adj_block_elem = tf.ones(Adj_block_idx.shape[1])
        
        #Add self-loops in the adjacency matrix if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.
        if not self.learn_eps:
            num_node = start_idx[-1]  #This is the number of nodes in the entire graph            
            self_loop_edge = tf.constant([range(num_node),range(num_node)])
            elem = tf.ones(num_node)
            Adj_block_idx = tf.concat([Adj_block_idx, self_loop_edge], 1)   #Adding connections from self-connections to the list of other connections specified
            Adj_block_elem = tf.concat([Adj_block_elem, elem], 0)  #Total number of connections + number of nodes
            
        Adj_block_idx = tf.cast(tf.transpose(Adj_block_idx),tf.int64)
        Adj_block = tf.SparseTensor(indices=Adj_block_idx, values=Adj_block_elem, dense_shape=[start_idx[-1],start_idx[-1]])        

        return Adj_block


    def __preprocess_graphpool(self, batch_graph):
        ###create sum or average pooling sparse matrix over entire nodes in each graph (num graphs x num nodes)
        
        start_idx = [0]

        #compute the padded neighbor list
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))

        idx = []
        elem = []
        for i, graph in enumerate(batch_graph):
            ###average pooling
            if self.graph_pooling_type == "average":
                elem.extend([1./len(graph.g)]*len(graph.g))
            #By default, it goes to the else
            else:
            ###sum pooling
                elem.extend([1]*len(graph.g))

            idx.extend([[i, j] for j in range(start_idx[i], start_idx[i+1], 1)])  #idx will be [[0,0],[0,1]...[0,218<end of the 1st graph>],[1,219<end of the first graph + 1>]...]
        
        elem = tf.constant(elem)        
        graph_pool = tf.SparseTensor(indices=idx, values=elem, dense_shape=[len(batch_graph),start_idx[-1]])
        graph_pool = tf.cast(graph_pool,tf.float32)        #graph_pool is a diagonal matrix of ones, where the ones are rows corresponding to the length of each graph.
        return graph_pool
    
    
    
    
    def maxpool(self, h, padded_neighbor_list):
        ###Element-wise minimum will never affect max-pooling

        dummy = tf.reduce_min(tf_output,axis=0,keepdims=True)
        h_with_dummy = tf.concat([tf_output,tf_dummy],0)        
        pooled_rep = tf.reduce_max(h_with_dummy[padded_neighbor_list], axis = 1)
        
        return pooled_rep


    def next_layer_eps(self, h, layer, padded_neighbor_list = None, Adj_block = None):
        ###pooling neighboring nodes and center nodes separately by epsilon reweighting. 

        if self.neighbor_pooling_type == "max":
            ##If max pooling
            pooled = self.maxpool(h, padded_neighbor_list)
        else:
            #If sum or average pooling
            h2 = tf.cast(h,tf.float32)
            pooled = tf.sparse.sparse_dense_matmul(Adj_block,h2)

            if self.neighbor_pooling_type == "average":  #The default is sum
                #If average pooling                
                degree = tf.sparse.sparse_dense_matmul(Adj_block,tf.ones([Adj_block.shape[0],1]))
                pooled = pooled/degree
       
        #Reweights the center node representation when aggregating it with its neighbors
        pooled = pooled + (1 + self.eps[layer])*h2        
        pooled_rep = self.mlps[layer](pooled)
        
        
        h = self.batches[layer](pooled_rep)        
        h = tf.nn.relu(h)
        h = tf.cast(h,tf.float32)
        
        
        return h


    def next_layer(self, h, layer, padded_neighbor_list = None, Adj_block = None):
        ###pooling neighboring nodes and center nodes altogether  
            
        if self.neighbor_pooling_type == "max":
            ##If max pooling
            pooled = self.maxpool(h, padded_neighbor_list)
        else:
            #If sum or average pooling            
            pooled = tf.sparse.sparse_dense_matmul(Adj_block,h)
            if self.neighbor_pooling_type == "average":
                #If average pooling
                degree = tf.sparse.sparse_dense_matmul(Adj_block,tf.ones([Adj_block.shape[0],1]))                
                pooled = pooled/degree

        #representation of neighboring and center nodes        
        pooled_rep = self.mlps[layer](pooled)
        
        h = self.batches[layer](pooled_rep)
        
        #non-linearity
        h = tf.nn.relu(h)        
        return h
      
    
    def call(self, batch_graph):
        
        
        X_concat = tf.concat([graph.node_features for graph in batch_graph],0)
        
        graph_pool = self.__preprocess_graphpool(batch_graph)

        if self.neighbor_pooling_type == "max":
            padded_neighbor_list = self.__preprocess_neighbors_maxpool(batch_graph)
        else:
            Adj_block = self.__preprocess_neighbors_sumavepool(batch_graph)
        
        #list of hidden representation at each layer (including input)
        hidden_rep = [X_concat]
        h = X_concat
        
        
        for layer in range(self.num_layers-1):
            if self.neighbor_pooling_type == "max" and self.learn_eps:
                h = self.next_layer_eps(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and self.learn_eps:  #This is the one that triggers for the reddit dataset
                h = self.next_layer_eps(h, layer, Adj_block = Adj_block)
            elif self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, Adj_block = Adj_block)

        
            hidden_rep.append(h)

        score_over_layer = 0
    
        #perform pooling over all nodes in each graph in every layer
        for layer, h in enumerate(hidden_rep):        
            h = tf.cast(h,tf.float32)
            pooled_h = tf.sparse.sparse_dense_matmul(graph_pool,h)            
            linear_outcome = self.linears[layer](pooled_h)
            dropped_outcome = self.drops[layer](linear_outcome)
            score_over_layer += dropped_outcome

        return score_over_layer  #This actually provides the value for the prediction

In [36]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)



graphs, num_classes = load_data(args.dataset, args.degree_as_tag)
train_graphs, test_graphs = separate_data(graphs, args.seed, args.fold_idx)
labels = tf.constant([graph.label for graph in train_graphs])

model = GraphCNN(args.num_layers, args.num_mlp_layers, args.hidden_dim, num_classes, args.final_dropout, args.learn_eps, args.graph_pooling_type, args.neighbor_pooling_type)

optimizer = tf.keras.optimizers.Adam(lr = args.lr)



#def train(loss,model,opt,original):    
def train(args,model,train_graphs,opt,epoch):
  total_iters = args.iters_per_epoch
  pbar = tqdm(range(total_iters),unit = 'batch')
  
  loss_accum = 0
  for pos in pbar:
    selected_idx = np.random.permutation(len(train_graphs))[:args.batch_size]
    batch_graph = [train_graphs[idx] for idx in selected_idx]
    labels = tf.constant([graph.label for graph in batch_graph])
    loss_accum = 0
    with tf.GradientTape() as tape:
      output = model(batch_graph)

      loss = loss_object(labels,output)

      
    gradients = tape.gradient(loss,model.trainable_variables)
    gradient_variables = zip(gradients, model.trainable_variables)
    opt.apply_gradients(gradient_variables)
    loss_accum += loss
    
    #report
    pbar.set_description(f'epoch: {epoch}')
    
  average_loss = loss_accum/total_iters
  print(f'loss training: {average_loss}')
  return average_loss


###pass data to model with minibatch during testing to avoid memory overflow (does not perform backpropagation)
def pass_data_iteratively(model, graphs, minibatch_size = 64):
    output = []
    idx = np.arange(len(graphs))
    for i in range(0, len(graphs), minibatch_size):
        sampled_idx = idx[i:i+minibatch_size]
        if len(sampled_idx) == 0:
            continue    
        output.append(model([graphs[j] for j in sampled_idx]))
    return tf.concat(output,0)

  
def tf_check_acc(pred,labels):
    pred = tf.cast(pred,tf.int32)
    correct = tf.equal(pred,labels)
    answer = 0
    for element in correct:
      if element:
        answer +=1
    return answer
  
def test(args, model, train_graphs, test_graphs, epoch):
    output = pass_data_iteratively(model, train_graphs)
    pred = tf.argmax(output,1)
    labels = tf.constant([graph.label for graph in train_graphs])

    correct = tf_check_acc(pred,labels)
    acc_train = correct / float(len(train_graphs))

    output = pass_data_iteratively(model, test_graphs)
    pred = tf.argmax(output,1)
    labels = tf.constant([graph.label for graph in test_graphs])
    correct = tf_check_acc(pred,labels)
    acc_test = correct / float(len(test_graphs))

    print("accuracy train: %f test: %f" % (acc_train, acc_test))

    return acc_train, acc_test





loading data
# classes: 2
# maximum node tag: 566
# data: 2000


In [37]:
import time

In [38]:
start_time = time.time()

for epoch in range(1, args.epochs + 1):
    if epoch % 50 == 0:
      optimizer.lr = optimizer.lr * 0.5    
    print (optimizer.lr)
    avg_loss = train(args, model, train_graphs, optimizer, epoch)
    acc_train, acc_test = test(args, model, train_graphs, test_graphs, epoch)
    
    if not args.filename == "":
        with open(args.filename, 'w') as f:
            f.write("%f %f %f" % (avg_loss, acc_train, acc_test))
            f.write("\n")
    print("")

    print(model.eps)

end_time = time.time()

print('run time =', end_time - start_time)

  0%|          | 0/50 [00:00<?, ?batch/s]

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>


epoch: 1: 100%|██████████| 50/50 [00:06<00:00,  8.20batch/s]


loss training: 846.5170288085938


epoch: 2:   4%|▍         | 2/50 [00:00<00:03, 15.95batch/s]

...

loss training: 0.5740084648132324


epoch: 5:   4%|▍         | 2/50 [00:00<00:03, 13.42batch/s]

accuracy train: 0.553889 test: 0.580000

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([-0.14038578, -0.15040207, -0.15236174, -0.19775099], dtype=float32)>
<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>


epoch: 5: 100%|██████████| 50/50 [00:04<00:00, 10.87batch/s]


loss training: 0.09010719507932663
accuracy train: 0.697778 test: 0.700000

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([-0.14043519, -0.15051115, -0.15243906, -0.19786331], dtype=float32)>
run time = 38.65967535972595
